# Alireza Bolhassani
---

## Introduction

In [58]:
import pandas as pd

groundtruth = pd.read_csv("groundtruth.tsv", delimiter="\t")
participants = pd.read_csv("participants.tsv", delimiter="\t")
groundtruth.head(3) 

,user_id,ad_clicked,attention,log_id
0,5npsk114ba8hfbj4jr3lt8jhf5,0,4,20181002033126
1,5o9js8slc8rg2a8mo5p3r93qm0,1,5,20181001211223
2,pi17qjfqmnhpsiahbumcsdq0r6,0,4,20181001170952


In [59]:
participants.head(3)

,user_id,country,education,age,income,gender,ad_position,ad_type,ad_category,serp_id,query,log_id
0,5npsk114ba8hfbj4jr3lt8jhf5,PHL,3,3,1,male,top-left,dd,Computers & Electronics,tablets,tablets,20181002033126
1,5o9js8slc8rg2a8mo5p3r93qm0,VEN,3,1,1,male,top-right,dd,Shop - Luxury Goods,casio-watches,casio watches,20181001211223
2,pi17qjfqmnhpsiahbumcsdq0r6,VEN,2,3,1,male,top-left,native,Shop - Luxury Goods,chivas-regal,chivas regal,20181001170952


---
## Merge two data frame

In [64]:
df = participants.merge(groundtruth,on=["user_id","log_id"])
df.head(3)

,user_id,country,education,age,income,gender,ad_position,ad_type,ad_category,serp_id,query,log_id,ad_clicked,attention
0,5npsk114ba8hfbj4jr3lt8jhf5,PHL,3,3,1,male,top-left,dd,Computers & Electronics,tablets,tablets,20181002033126,0,4
1,5o9js8slc8rg2a8mo5p3r93qm0,VEN,3,1,1,male,top-right,dd,Shop - Luxury Goods,casio-watches,casio watches,20181001211223,1,5
2,pi17qjfqmnhpsiahbumcsdq0r6,VEN,2,3,1,male,top-left,native,Shop - Luxury Goods,chivas-regal,chivas regal,20181001170952,0,4


---
## Filter based on time spent

In [56]:
import os
df["time_spent"] = 0

for index,row in df.iterrows():
    file_path = f"logs/{row['log_id_x']}.csv"
    if not os.path.exists(file_path):
        continue

    df2 = pd.read_csv(file_path, delimiter=" ")
    df.at[index,"time_spent"] = df2["timestamp"].iloc[-1] - df2["timestamp"].iloc[0]

df = df[df["time_spent"]>5000]
df.head(3)

,user_id,country,education,age,income,gender,ad_position,ad_type,ad_category,serp_id,query,log_id_x,ad_clicked,attention,log_id_y,time_spent
0,5npsk114ba8hfbj4jr3lt8jhf5,PHL,3,3,1,male,top-left,dd,Computers & Electronics,tablets,tablets,20181002033126,0,4,20181002033126,96342
1,5o9js8slc8rg2a8mo5p3r93qm0,VEN,3,1,1,male,top-right,dd,Shop - Luxury Goods,casio-watches,casio watches,20181001211223,1,5,20181001211223,18751
2,pi17qjfqmnhpsiahbumcsdq0r6,VEN,2,3,1,male,top-left,native,Shop - Luxury Goods,chivas-regal,chivas regal,20181001170952,0,4,20181001170952,54203


---


In [67]:
df["ad_category"].unique()

array(['Computers & Electronics', 'Shop - Luxury Goods',
       'Autos & Vehicles', 'Travel', 'Games',
       'Shop - Photo & Video Services', 'Shop - Toys',
       'Shop - Sporting Goods', 'Shop - Apparel',
       'Shop - Wholesalers & Liquidatr', 'Shop - Gifts & Special Event',
       'Shop - Event Ticket Sales', 'Real Estate', 'Food & Drink'],
      dtype=object)